In [9]:
import numpy as np
from scipy.integrate import solve_ivp

In [13]:
def r2bp(t, y):
    rvect = y[0:3].copy()
    vvect = y[3:6].copy()
    r = np.linalg.norm(rvect)
    dydt = np.append(vvect, -rvect/r**3)
    return dydt

In [33]:
def events(s, z, tf):
    return z[12] - tf

In [30]:
def r2bp_sb(s, z):
    rvect = z[0:3].copy()
    wvect = z[3:6].copy()
    r = z[6]
    q = z[7]
    h = z[8]
    Avect = z[9:12].copy()
    t = z[12]
    dzds = np.empty((13,))
    dzds[0:3] = wvect
    dzds[3:6] = 2*h*rvect - Avect
    dzds[6] = q
    dzds[7] = 2*h*r + 1
    dzds[8] = 0
    dzds[9:12] = np.array([0.0, 0.0, 0.0])
    dzds[12] = r
    return dzds

In [40]:
t0 = 0.0
tf = 10*2*np.pi


rvect0 = np.array([1.0, 0.0, 0.0])
vvect0 = np.array([0.0, 1.0, 0.0])
r0 = np.linalg.norm(rvect0)
v0 = np.linalg.norm(vvect0)

y0 = np.append(rvect0, vvect0)

s0 = 0.0
sf = 1000.0

wvect0 = r0*rvect0
q0 = np.dot(rvect0, vvect0)
h0 = v0**2/2 - 1/r0
Avect0 = -rvect0/r0 + np.cross(vvect0, np.cross(rvect0, vvect0))
z0 = np.concatenate((rvect0, wvect0, [r0], [q0], [h0], Avect0, [t0]))

events_r = lambda s, z: events(s, z, tf)
events_r.terminal = True
events_r.direction = +1

In [41]:
tol = 1e-9
method = 'RK45'
sol = solve_ivp(r2bp, [t0, tf], y0, atol=tol, rtol=tol, method=method)
sol_sb = solve_ivp(r2bp_sb, [s0, sf], z0, events=events_r, atol=tol, rtol=tol, method=method)
print(np.linalg.norm(sol.y[0:3, -1] - sol.y[0:3, 0]))
print(np.linalg.norm(sol_sb.y[0:3, -1] - sol_sb.y[0:3, 0]))

print(sol.nfev)
print(sol_sb.nfev)

2.9012762723267594e-06
4.4075100680274204e-08
5048
4820
